In [2]:
import pickle
import pandas as pd
import math
from scipy import stats
import seaborn as sns
import random
from sklearn.utils import shuffle

resources get initialised


In [6]:
datasets = ['ASSET', 'AutoMeTS', 'BenchLS', 'britannica', 'EW-SEW-Turk', 'HutSSF', 'metaeval', 'MTurkSF', 'NNSeval', 'OneStopEnglish', 'QuestEval', 'SemEval_2007', 'SimpEval_22', 'simpa', 'TurkCorpus']

for ds in datasets:
    load_data(ds)

In [5]:
def load_data(d_s):
    simp = []
    src = []

    simp_path = f"/workspace/datasets/ds_labels/{d_s}-1249_simp_labels.pkl"        
    src_path = f"/workspace/datasets/ds_labels/{d_s}-1249_src_labels.pkl"  
    
    simp_labels = pickle.load(open(simp_path, "rb"))
    src_labels = pickle.load(open(src_path, "rb")) 

    for entry in simp_labels:
        src.append([number for number in entry.tolist()])

    for entry in src_labels:
        simp.append([number for number in entry.tolist()])
        
    # load textual data
    if d_s == 'EW-SEW-Turk':
        textual_dataset = pd.read_pickle('/workspace/datasets/ewsewturk/ewsewturk.pkl')
    else: 
        if d_s == 'SemEval_2007':    
            textual_dataset = pd.read_pickle('/workspace/datasets/semeval07/semeval07.pkl')
        else:
            if d_s == 'SimpEval_22':
                textual_dataset = pd.read_pickle('/workspace/datasets/simpeval/simpeval.pkl')
            else:
                textual_dataset = pd.read_pickle('/workspace/datasets/' + d_s + '/' + d_s + '.pkl')

    simp_dup = ~textual_dataset['simp'].duplicated()
    src_dup = ~textual_dataset['src'].duplicated()

    dups = simp_dup & src_dup

    data_merged_simp_dedup = []
    data_merged_src_dedup = []

    for index, val in dups.items():
        if val:
            data_merged_simp_dedup.append(simp[index])
            data_merged_src_dedup.append(src[index])

    X = data_merged_simp_dedup + data_merged_src_dedup

    # pickle
    with open('/workspace/vectors/' + ds + '_vectors.pkl', 'wb') as f:
        pickle.dump(X, f)


In [14]:
def load_ARTS_data(dims):
    scores = []
    embs = []
    labels = []

    if dims == 94:
        path_X = f"/workspace/datasets/ARTS/arts94-1249_labels.pkl"        
        path_e = f"/workspace/datasets/ARTS/ARTS_94_small_embeddings.pkl"
        path_s = f"/workspace/datasets/ARTS/Gold_Scores.pkl"      
        path_label = f"/workspace/datasets/ARTS/ARTS_94_DF.pkl"      

    if dims == 300:
        path_X = f"/workspace/datasets/ARTS/arts300-1249_labels.pkl"        
        path_e = f"/workspace/datasets/ARTS/ARTS_300_small_embeddings.pkl"
        path_s = f"/workspace/datasets/ARTS/gpt-4-1106-preview-300_scores.pkl"  
        path_label = f"/workspace/datasets/ARTS/ARTS_300_DF.pkl"      
    
    if dims == 3000:
        path_X = f"/workspace/datasets/ARTS/arts3000-1249_labels.pkl"    
        path_e = f"/workspace/datasets/ARTS/ARTS_3000_small_embeddings.pkl"
        path_s = f"/workspace/datasets/ARTS/gpt-4-1106-preview-3000_scores.pkl" 
        path_label = f"/workspace/datasets/ARTS/ARTS_3000_DF.pkl"      
 

    X = pickle.load(open(path_X, "rb"))
    embeddings = pickle.load(open(path_e, 'rb'))['gpt_embedding']
    cur_scores = pickle.load(open(path_s, "rb"))

    for dp in embeddings:
       embs.append(dp)

    for i in range(len(X)):
        scores.append(cur_scores[i][0])

    cur_labels = pickle.load(open(path_label, 'rb'))

    for i, r in cur_labels.iterrows():
        labels.append(r['Label'])

    # pickle
    with open('/workspace/vectors/ARTS_' + str(dims) + '_vectors.pkl', 'wb') as f:
        pickle.dump((X, scores, embs, labels), f)


In [15]:
load_ARTS_data(94)
load_ARTS_data(300)
load_ARTS_data(3000)

In [17]:
def load_200_data(datasets, name):
    all_simp = []
    all_src = []

    for d_s in datasets:    
        simp = []
        src = []
       
        simp_path = f"/workspace/datasets/ds_labels/{d_s}-1249_simp_labels.pkl"        
        src_path = f"/workspace/datasets/ds_labels/{d_s}-1249_src_labels.pkl"  
        
        simp_labels = pickle.load(open(simp_path, "rb"))
        src_labels = pickle.load(open(src_path, "rb"))

        for entry in simp_labels:
            simp.append([number for number in entry.tolist()])

        for entry in src_labels:
            src.append([number for number in entry.tolist()])

        # load textual data
        if d_s == 'EW-SEW-Turk':
            textual_dataset = pd.read_pickle('/workspace/datasets/ewsewturk/ewsewturk.pkl')
        else: 
            if d_s == 'SemEval_2007':    
                textual_dataset = pd.read_pickle('/workspace/datasets/semeval07/semeval07.pkl')
            else:
                if d_s == 'SimpEval_22':
                    textual_dataset = pd.read_pickle('/workspace/datasets/simpeval/simpeval.pkl')
                else:
                    textual_dataset = pd.read_pickle('/workspace/datasets/' + d_s + '/' + d_s + '.pkl')
            
        simp_dup = ~textual_dataset['simp'].duplicated()
        src_dup = ~textual_dataset['src'].duplicated()

        dups = simp_dup & src_dup

        found = 0

        random.seed(42)

        dups = shuffle(dups, random_state=42)


        for index, val in dups.items():
            if val and found < 200:
                found += 1
                all_simp.append(simp[index])
                all_src.append(src[index])

    X = all_simp + all_src

    # pickle
    with open('/workspace/vectors/' + name + '_vectors.pkl', 'wb') as f:
        pickle.dump(X, f)


In [18]:
datasets = [['britannica', 'TurkCorpus'], ['NNSeval', 'OneStopEnglish', 'simpa', 'TurkCorpus'], ['HutSSF', 'OneStopEnglish'], ['britannica', 'EW-SEW-Turk', 'metaeval', 'NNSeval', 'SimpEval_22', 'TurkCorpus']]
names = ['TA_children', 'TA_language', 'D_news', 'D_encyclopedia']


for ds in range(len(datasets)):
    
    load_200_data(datasets[ds], names[ds])